In [1]:
import sys
import os

from IPython import display
from time import time
from collections import Counter
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

print(os.getcwd())
inputPath = 'C:\\Users\\idstwy\\Desktop\\SSHSPH\\SPH6004 Advanced Statistical Learning\\Assignment\\Group\\Data'

F:\SPH6004 Advanced Statistical Learning\Assignment\Group\Codes


# read data files

In [2]:
#==============
#read data files
#==============
#medication
medRXcui= pd.read_excel(inputPath + "\\Med_Rxcui.xlsx", sheet_name="Med2RxCUI", header=0)
medRXcui = medRXcui.drop_duplicates()
print(medRXcui.shape)

#Patient 
d1 = pd.read_csv(inputPath + "\\SSHSPH_AdvBioStat_AKI\\patient.csv", index_col=None, parse_dates=['aki_charttime', "icu_intime"])
d1.columns = [c.strip().replace(" ", "_") for c in d1.columns.values.tolist()]
print(d1.shape)
print(list(d1.columns))

#Patient_timeseries 
ts1 = pd.read_csv(inputPath + "\\SSHSPH_AdvBioStat_AKI\\TimeSeries.csv", index_col=None, parse_dates=['charttime'])
ts1.columns = [c.strip().replace(" ", "_") for c in ts1.columns.values.tolist()]
ts1 = ts1.drop(columns=['specimen', 'specimen_pred', 'specimen_prob'])
print(ts1.shape)
print(list(ts1.columns))

(329, 2)
(21164, 24)
['stay_id', 'icu_intime', 'aki_charttime', 'adm_age', 'weight', 'age_score', 'myocardial_infarct', 'congestive_heart_failure', 'peripheral_vascular_disease', 'cerebrovascular_disease', 'dementia', 'chronic_pulmonary_disease', 'rheumatic_disease', 'peptic_ulcer_disease', 'mild_liver_disease', 'diabetes_without_cc', 'diabetes_with_cc', 'paraplegia', 'renal_disease', 'malignant_cancer', 'severe_liver_disease', 'metastatic_solid_tumor', 'aids', 'charlson_comorbidity_index']
(570653, 35)
['stay_id', 'charttime', 'medication', 'so2', 'po2', 'pco2', 'fio2_chartevents', 'fio2', 'aado2', 'aado2_calc', 'pao2fio2ratio', 'ph', 'baseexcess', 'bicarbonate', 'totalco2', 'hematocrit', 'hemoglobin', 'carboxyhemoglobin', 'methemoglobin', 'chloride', 'calcium', 'temperature', 'potassium', 'sodium', 'lactate', 'glucose', 'heart_rate_avg', 'sbp_avg', 'dbp_avg', 'sbp_ni_avg', 'dbp_ni_avg', 'mbp_ni_avg', 'tmp_avg', 'spo2_avg', 'glucose_avg']


# Time series data (collapse 3 hourly interval)

In [3]:
IDList = list(ts1["stay_id"].unique())
IDList[0]

appendData=[]
for i in IDList:
    STAYID = i
    d_temp = ts1[ts1["stay_id"]==STAYID]
    minCharttime = d_temp["charttime"].min()
    d_temp = d_temp.sort_values(by=["stay_id", "charttime"])
    d_temp = d_temp.set_index(["charttime"])
    d_temp = d_temp.drop(columns=["stay_id"])
    d_temp1 = d_temp.resample('3H', origin=minCharttime).mean()
    d_temp1 = d_temp1.reset_index()
    d_temp1["stay_id"] = STAYID
    d_temp1.insert(0, "stay_id", d_temp1.pop("stay_id"))
    d_temp1 = d_temp1.ffill()
    d_temp1 = d_temp1.bfill()
    appendData.append(d_temp1)
    
TS_chart = pd.concat(appendData)
print(TS_chart.shape)
TS_chart.head(3)

(169305, 34)


,stay_id,charttime,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,bicarbonate,totalco2,hematocrit,hemoglobin,carboxyhemoglobin,methemoglobin,chloride,calcium,temperature,potassium,sodium,lactate,glucose,heart_rate_avg,sbp_avg,dbp_avg,sbp_ni_avg,dbp_ni_avg,mbp_ni_avg,tmp_avg,spo2_avg,glucose_avg
0,30001535,2177-11-17 21:00:00,NaN,132.0,28.0,40.0,NaN,NaN,118.2,330.0,7.38,-6.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,1.08,NaN,NaN,NaN,3.0,NaN,108.5,102.500000,77.0,102.5,77.0,83.0,36.833333,99.5,140.0
0,30002052,2180-12-29 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,117.666667,61.0,109.0,58.0,74.0,36.777778,95.5,164.0
1,30002052,2180-12-29 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,117.666667,61.0,109.0,58.0,74.0,36.777778,95.5,164.0


# Medications

In [51]:
dMed = ts1[["stay_id", "charttime", "medication"]]
dMed= dMed.dropna()
dMed = dMed.drop_duplicates()

medRXcui1 = medRXcui[["medication", "MedCat"]].drop_duplicates()
dMed = dMed.merge(medRXcui1, on="medication", how="inner")
dMed["MedCat"] = dMed["MedCat"].apply(lambda x: "Med_"+str(x))
dMed = dMed[~dMed["MedCat"].isin(['Med_nan', 'Med_None'])]
dMed = dMed.drop_duplicates()
dMed = dMed.sort_values(by=["stay_id", "charttime"])
dMed = dMed[["stay_id", "charttime", "MedCat"]].drop_duplicates()
dMed = dMed.reset_index(drop=True)
dMed["flag"] = 1
print(dMed['MedCat'].value_counts())

dMed1 = pd.pivot_table(dMed, index= ["stay_id", "charttime"], columns='MedCat', values='flag')
dMed1 = dMed1.sort_values(by=["stay_id", "charttime"], ascending=True)
dMed1 = dMed1.reset_index() 
dMed1 = dMed1.fillna(0)

IDList = list(dMed1["stay_id"].unique())
IDList[0]

appendData=[]
for i in IDList:
    STAYID = i
    d_temp = dMed1[dMed1["stay_id"]==STAYID]
    minCharttime = d_temp["charttime"].min()
    d_temp = d_temp.sort_values(by=["stay_id", "charttime"])
    d_temp = d_temp.set_index(["charttime"])
    d_temp = d_temp.drop(columns=["stay_id"])
    d_temp1 = d_temp.resample('3H', origin=minCharttime).max()
    d_temp1 = d_temp1.reset_index()
    d_temp1["stay_id"] = STAYID
    d_temp1.insert(0, "stay_id", d_temp1.pop("stay_id"))
    d_temp1 = d_temp1.fillna(0)
    appendData.append(d_temp1)
    
TS_med = pd.concat(appendData)
print(TS_med.shape)
TS_med.head(3)

Med_NSAID              10046
Med_Vancomycin          8484
Med_Diuretics           6960
Med_Angiotensin         2605
Med_Aminoglycosides      125
Name: MedCat, dtype: int64
(40395, 7)


MedCat,stay_id,charttime,Med_Aminoglycosides,Med_Angiotensin,Med_Diuretics,Med_NSAID,Med_Vancomycin
0,30002095,2190-07-30 18:37:00,0.0,1.0,0.0,0.0,0.0
1,30002095,2190-07-30 21:37:00,0.0,0.0,0.0,0.0,0.0
2,30002095,2190-07-31 00:37:00,0.0,0.0,0.0,0.0,0.0


# Demo Data

In [52]:
#each patient can have >1 specimen_pred and specimen_prob!!!

#columns related to charlson_comorbidity_index 
CCI = ['age_score', 'myocardial_infarct', 'congestive_heart_failure', 'peripheral_vascular_disease', 'cerebrovascular_disease', 'dementia', 
       'chronic_pulmonary_disease', 'rheumatic_disease', 'peptic_ulcer_disease', 'mild_liver_disease', 'diabetes_without_cc', 'diabetes_with_cc', 
       'paraplegia', 'renal_disease', 'malignant_cancer', 'severe_liver_disease', 'metastatic_solid_tumor', 'aids', ] #'age_score', 'specimen_pred', 'specimen_prob'

demo = ['stay_id', 'icu_intime', 'aki_charttime', 'adm_age', 'weight', 'charlson_comorbidity_index']
demo_df = d1[demo+CCI].drop_duplicates()
print(demo_df.shape)
demo_df.head(3)

(21164, 24)


,stay_id,icu_intime,aki_charttime,adm_age,weight,charlson_comorbidity_index,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,peptic_ulcer_disease,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids
0,36691552,2150-12-09 16:59:14,NaT,66,66.0,8,3,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1,36782431,2191-12-16 17:51:05,2191-12-17 12:00:00,73,80.0,9,4,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,37377877,2180-07-11 18:13:00,2180-07-12 11:00:00,87,90.0,11,4,0,1,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0


# Merge

In [53]:
d_master = demo_df.merge(TS_chart, on="stay_id", how="outer")
d_master = d_master.merge(TS_med, on=["stay_id", "charttime"], how="left")
d_master.insert(3, "charttime", d_master.pop("charttime"))
d_master = d_master.sort_values(by=["stay_id", "icu_intime", "charttime"])
print(d_master.shape)
d_master.head(3)

(169305, 62)


,stay_id,icu_intime,aki_charttime,charttime,adm_age,weight,charlson_comorbidity_index,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,peptic_ulcer_disease,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,bicarbonate,totalco2,hematocrit,hemoglobin,carboxyhemoglobin,methemoglobin,chloride,calcium,temperature,potassium,sodium,lactate,glucose,heart_rate_avg,sbp_avg,dbp_avg,sbp_ni_avg,dbp_ni_avg,mbp_ni_avg,tmp_avg,spo2_avg,glucose_avg,Med_Aminoglycosides,Med_Angiotensin,Med_Diuretics,Med_NSAID,Med_Vancomycin
32458,30001535,2177-11-17 20:30:00,2177-11-17 23:51:00,2177-11-17 21:00:00,69,80.0,6,3,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN,132.0,28.0,40.0,NaN,NaN,118.2,330.0,7.38,-6.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,1.08,NaN,NaN,NaN,3.0,NaN,108.5,102.500000,77.0,102.5,77.0,83.0,36.833333,99.5,140.0,NaN,NaN,NaN,NaN,NaN
66051,30002052,2180-12-30 05:24:46,2180-12-31 10:00:00,2180-12-29 08:00:00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,117.666667,61.0,109.0,58.0,74.0,36.777778,95.5,164.0,NaN,NaN,NaN,NaN,NaN
66052,30002052,2180-12-30 05:24:46,2180-12-31 10:00:00,2180-12-29 11:00:00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,117.666667,61.0,109.0,58.0,74.0,36.777778,95.5,164.0,NaN,NaN,NaN,NaN,NaN


# Create time variables & Select Records

In [54]:
d_master2 = d_master.copy()
d_master2["t_ICU2Chart"] = (d_master2["charttime"] - d_master2["icu_intime"]) / pd.Timedelta(hours=1)
d_master2["t_ICU2AKI"] = (d_master2["aki_charttime"] - d_master2["icu_intime"]) / pd.Timedelta(hours=1)
d_master2["t_ICU2AKI"] = d_master2["t_ICU2AKI"].fillna(0)
d_master2["t_Chart2AKI"] = (d_master2["aki_charttime"] - d_master2["charttime"]) / pd.Timedelta(hours=1)

#to exclude records if chart time before ICU intime 
d_master2 = d_master2[d_master2["t_ICU2Chart"]>=0]
d_master2 = d_master2.reset_index(drop=True)

d_master2.insert(3, "t_Chart2AKI", d_master2.pop("t_Chart2AKI"))
d_master2.insert(3, "t_ICU2AKI", d_master2.pop("t_ICU2AKI"))
d_master2.insert(3, "t_ICU2Chart", d_master2.pop("t_ICU2Chart"))
d_master2.insert(3, "charttime", d_master2.pop("charttime"))
d_master2.insert(3, "aki_charttime", d_master2.pop("aki_charttime"))

print(d_master2.shape)
d_master2.head()

(123257, 65)


,stay_id,icu_intime,charttime,aki_charttime,t_ICU2Chart,t_ICU2AKI,t_Chart2AKI,adm_age,weight,charlson_comorbidity_index,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,peptic_ulcer_disease,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,bicarbonate,totalco2,hematocrit,hemoglobin,carboxyhemoglobin,methemoglobin,chloride,calcium,temperature,potassium,sodium,lactate,glucose,heart_rate_avg,sbp_avg,dbp_avg,sbp_ni_avg,dbp_ni_avg,mbp_ni_avg,tmp_avg,spo2_avg,glucose_avg,Med_Aminoglycosides,Med_Angiotensin,Med_Diuretics,Med_NSAID,Med_Vancomycin
0,30001535,2177-11-17 20:30:00,2177-11-17 21:00:00,2177-11-17 23:51:00,0.500000,3.350000,2.85,69,80.0,6,3,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN,132.0,28.0,40.0,NaN,NaN,118.2,330.0,7.38,-6.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,1.08,NaN,NaN,NaN,3.0,NaN,108.5,102.500000,77.000000,102.50,77.000000,83.0,36.833333,99.50,140.0,NaN,NaN,NaN,NaN,NaN
1,30002052,2180-12-30 05:24:46,2180-12-30 08:00:00,2180-12-31 10:00:00,2.587222,28.587222,26.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.5,122.833333,69.500000,121.00,69.750000,84.5,36.555556,96.75,164.0,NaN,NaN,NaN,NaN,NaN
2,30002052,2180-12-30 05:24:46,2180-12-30 11:00:00,2180-12-31 10:00:00,5.587222,28.587222,23.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,123.000000,70.333333,123.00,70.333333,85.5,36.555556,98.00,164.0,NaN,NaN,NaN,NaN,NaN
3,30002052,2180-12-30 05:24:46,2180-12-30 14:00:00,2180-12-31 10:00:00,8.587222,28.587222,20.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,116.750000,75.500000,116.75,75.500000,89.0,36.555556,96.25,149.0,NaN,NaN,NaN,NaN,NaN
4,30002052,2180-12-30 05:24:46,2180-12-30 17:00:00,2180-12-31 10:00:00,11.587222,28.587222,17.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,129.000000,91.000000,129.00,91.000000,102.0,36.555556,96.00,149.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# dTemp1  =pd.DataFrame(d1ts1.groupby(["stay_id"]).agg({"icu_intime": "min", "charttime": "max", "aki_charttime": "max"}).reset_index())
# dTemp1["MaxTime"] = dTemp1[["charttime", 'aki_charttime']].max(axis=1)
# dTemp1['LOS'] = (dTemp1["MaxTime"] - dTemp1["icu_intime"]) / pd.Timedelta(hours=1)
# dTemp1["newVariable"] = np.where(dTemp1["aki_charttime"].isnull(), dTemp1['LOS'], dTemp1['t_ICU2AKI'])
# dTemp1.head()

In [55]:
#==================
#select patients
#===================
d1a = d1[["stay_id", 'aki_charttime', "icu_intime"]]
ts1a = ts1[["stay_id", "charttime"]]
d1ts1 =d1a.merge(ts1a, on="stay_id", how='outer') 
d1ts1 = d1ts1.drop_duplicates()
print(d1ts1.shape)

#==============
#select only IDs with more than 2 LOS in ICU
#==============
dTemp1  =pd.DataFrame(d1ts1.groupby(["stay_id"]).agg({"icu_intime": "min", "charttime": "max", "aki_charttime": "max"}).reset_index())
dTemp1["MaxTime"] = dTemp1[["charttime", 'aki_charttime']].max(axis=1)
dTemp1['LOS'] = (dTemp1["MaxTime"] - dTemp1["icu_intime"]) / pd.Timedelta(hours=1)
print(dTemp1['LOS'].describe())
dTemp1.head()

#consider IDs with 10>hrs or more in ICU stay
#98%(20841/21164) of patients have  have at least 2 hours or more LOS in ICU
listID_GTE2hrs = dTemp1[dTemp1['LOS']>=2]["stay_id"]
listID_GTE2hrs = list(set(listID_GTE2hrs))
print("")
print("Nbr of IDs with 10 hours or more stay in ICUs:", len(listID_GTE2hrs))

d_master3 = d_master2[d_master2["stay_id"].isin(listID_GTE2hrs)]
print(d_master3.shape)

(472162, 4)
count    21164.000000
mean        18.224394
std          8.416130
min         -5.443333
25%         11.317708
50%         20.209722
75%         23.408889
max         48.000000
Name: LOS, dtype: float64

Nbr of IDs with 10 hours or more stay in ICUs: 20841
(123138, 65)


In [56]:
medCols = [i for i in d_master3.columns if i.startswith("Med_")]
for i in medCols:
    d_master3[i]= d_master3[i].fillna(0)

In [57]:
print(d_master3.shape)
print(list(d_master3.columns))
d_master3.to_csv(inputPath + "\\d_master_3hourly.csv", index=False)

(123138, 65)
['stay_id', 'icu_intime', 'charttime', 'aki_charttime', 't_ICU2Chart', 't_ICU2AKI', 't_Chart2AKI', 'adm_age', 'weight', 'charlson_comorbidity_index', 'age_score', 'myocardial_infarct', 'congestive_heart_failure', 'peripheral_vascular_disease', 'cerebrovascular_disease', 'dementia', 'chronic_pulmonary_disease', 'rheumatic_disease', 'peptic_ulcer_disease', 'mild_liver_disease', 'diabetes_without_cc', 'diabetes_with_cc', 'paraplegia', 'renal_disease', 'malignant_cancer', 'severe_liver_disease', 'metastatic_solid_tumor', 'aids', 'so2', 'po2', 'pco2', 'fio2_chartevents', 'fio2', 'aado2', 'aado2_calc', 'pao2fio2ratio', 'ph', 'baseexcess', 'bicarbonate', 'totalco2', 'hematocrit', 'hemoglobin', 'carboxyhemoglobin', 'methemoglobin', 'chloride', 'calcium', 'temperature', 'potassium', 'sodium', 'lactate', 'glucose', 'heart_rate_avg', 'sbp_avg', 'dbp_avg', 'sbp_ni_avg', 'dbp_ni_avg', 'mbp_ni_avg', 'tmp_avg', 'spo2_avg', 'glucose_avg', 'Med_Aminoglycosides', 'Med_Angiotensin', 'Med_Diu

In [60]:
#============================
# bring in procedure_Intub_Venti info
#============================
import pandas as pd
d_all = pd.read_csv(inputPath + "\\d_master_3hourly.csv", index_col=None, parse_dates=["icu_intime"])
proc1 = pd.read_csv(inputPath + "\\SSHSPH_AdvBioStat_AKI\\procedure_Intub_Venti.csv", index_col=None, parse_dates=["starttime"])
print(d_all["icu_intime"].min(),d_all["icu_intime"].max())
print(proc1["starttime"].min(),proc1["starttime"].max())

#bring in procedure_Intub_Venti info, consider ICU records wihtin intime date ranges 
select = (proc1["starttime"]>= d_all["icu_intime"].min()) & (proc1["starttime"]<= d_all["icu_intime"].max())
proc2 = proc1[select]
print(proc1.shape, proc2.shape)

2110-01-13 09:09:40 2210-09-08 21:41:50
2110-01-11 12:40:00 2211-05-08 13:07:00
(67977, 6) (67970, 6)


In [64]:
#============================
# Collapse procedure_Intub_Venti at patient level, yes if patient is Ventilated during the ICU stay, else No
#============================
import datetime
proc2["starttime_date"] = proc2["starttime"].apply(lambda x:x.date())
proc3=pd.DataFrame(proc2.groupby(["stay_id"]).agg({"Procedure_Ventilation": "sum", "Procedure_Intubation_Extubation": "sum"}).reset_index())
print(proc3.shape)
d_all1 = d_all.merge(proc3, on=["stay_id"], how='left')
d_all1["Procedure_Intubation_Extubation"] = np.where(d_all1["Procedure_Intubation_Extubation"]>0,1,0)
d_all1["Procedure_Ventilation"] = np.where(d_all1["Procedure_Ventilation"]>0,1,0)
print(d_all1.shape)
#d_all1.to_csv(inputPath + "\\d_master_3hourly.csv", index=False)

(31458, 3)
(123138, 67)


In [67]:
d_all1.head(10)

,stay_id,icu_intime,charttime,aki_charttime,t_ICU2Chart,t_ICU2AKI,t_Chart2AKI,adm_age,weight,charlson_comorbidity_index,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,peptic_ulcer_disease,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,so2,po2,pco2,fio2_chartevents,fio2,aado2,aado2_calc,pao2fio2ratio,ph,baseexcess,bicarbonate,totalco2,hematocrit,hemoglobin,carboxyhemoglobin,methemoglobin,chloride,calcium,temperature,potassium,sodium,lactate,glucose,heart_rate_avg,sbp_avg,dbp_avg,sbp_ni_avg,dbp_ni_avg,mbp_ni_avg,tmp_avg,spo2_avg,glucose_avg,Med_Aminoglycosides,Med_Angiotensin,Med_Diuretics,Med_NSAID,Med_Vancomycin,Procedure_Ventilation,Procedure_Intubation_Extubation
0,30001535,2177-11-17 20:30:00,2177-11-17 21:00:00,2177-11-17 23:51:00,0.500000,3.350000,2.85,69,80.0,6,3,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN,132.0,28.0,40.0,NaN,NaN,118.2,330.0,7.38,-6.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,1.08,NaN,NaN,NaN,3.0,NaN,108.50,102.500000,77.000000,102.50,77.000000,83.00,36.833333,99.50,140.0,0.0,0.0,0.0,0.0,0.0,0,0
1,30002052,2180-12-30 05:24:46,2180-12-30 08:00:00,2180-12-31 10:00:00,2.587222,28.587222,26.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.50,122.833333,69.500000,121.00,69.750000,84.50,36.555556,96.75,164.0,0.0,0.0,0.0,0.0,0.0,0,0
2,30002052,2180-12-30 05:24:46,2180-12-30 11:00:00,2180-12-31 10:00:00,5.587222,28.587222,23.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.00,123.000000,70.333333,123.00,70.333333,85.50,36.555556,98.00,164.0,0.0,0.0,0.0,0.0,0.0,0,0
3,30002052,2180-12-30 05:24:46,2180-12-30 14:00:00,2180-12-31 10:00:00,8.587222,28.587222,20.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.00,116.750000,75.500000,116.75,75.500000,89.00,36.555556,96.25,149.0,0.0,0.0,0.0,0.0,0.0,0,0
4,30002052,2180-12-30 05:24:46,2180-12-30 17:00:00,2180-12-31 10:00:00,11.587222,28.587222,17.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.00,129.000000,91.000000,129.00,91.000000,102.00,36.555556,96.00,149.0,0.0,0.0,0.0,0.0,0.0,0,0
5,30002052,2180-12-30 05:24:46,2180-12-30 20:00:00,2180-12-31 10:00:00,14.587222,28.587222,14.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.75,119.500000,71.000000,119.50,71.000000,85.50,36.555556,95.50,188.0,0.0,0.0,0.0,0.0,0.0,0,0
6,30002052,2180-12-30 05:24:46,2180-12-30 23:00:00,2180-12-31 10:00:00,17.587222,28.587222,11.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.00,103.500000,77.500000,103.50,77.500000,88.00,36.555556,95.00,188.0,0.0,0.0,0.0,0.0,0.0,0,0
7,30002052,2180-12-30 05:24:46,2180-12-31 02:00:00,2180-12-31 10:00:00,20.587222,28.587222,8.00,63,56.7,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.50,113.500000,70.000000,113.50,70.000000,81.75,36.500000,95.75,125.0,0.0,0.0,0.0,0.0,0.0,0,0
8,30002095,2190-07-30 14:35:16,2190-07-30 16:58:00,NaN,2.378889,0.000000,NaN,63,106.9,7,3,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,147.0,46.0,NaN,NaN,NaN,NaN,NaN,7.24,-7.0,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.50,132.000000,80.000000,132.00,80.000000,91.25,36.666667,95.50,195.0,0.0,0.0,0.0,0.0,0.0,0,0
9,30002095,2190-07-30 14:35:16,2190-07-30 19:58:00,NaN,5.378889,0.000000,NaN,63,106.9,7,3,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,147.0,46.0,Na